<a href="https://colab.research.google.com/github/raminnourizade/Proposal/blob/main/eeg_tourch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install mne

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 19.4 MB/s eta 0:00:00


In [2]:
import gdown
import os
import numpy as np
import pandas as pd
import mne
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import glob
from sklearn.preprocessing import LabelBinarizer
import tensorflow as tf
import keras
from tensorflow.keras import layers
import sys
os.environ["TF_CPP_MIN_LOG_LEVEL"]="3"
import matplotlib.pyplot as plt

from tensorflow.keras import layers, models, optimizers
from sklearn.utils import shuffle
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler
from tensorflow.keras.layers.experimental.preprocessing import RandomFlip, RandomRotation, RandomZoom
from scipy import signal

from tensorflow.keras import regularizers

from tensorflow.keras.layers import BatchNormalization
from sklearn.model_selection import KFold
from sklearn.model_selection import LeaveOneOut


In [14]:
def load_data():
  path="dataset"
  if not os.path.exists(path):
      os.mkdir(path)
      print(f"Folder {path} created!")

      file_id = "1hG5v_COjPNzejRaL9XJAFERee9i2_V04"  # Replace this with your file's ID
      output_file = path+"/eeg.zip"  # Replace "data_file.ext" with the desired output filename and extension
      gdown.download(f"https://drive.google.com/uc?id={file_id}", output_file)
      !unzip "dataset/eeg.zip" -d "dataset"
      os.remove("dataset/eeg.zip")

  else:
      print(f"Folder {path} already exists")


  ds_dir = '/content/dataset/'
  scaler = StandardScaler()
  all_eeg=[]

  for (i, item) in enumerate(glob.glob(ds_dir + '*.edf')):
    print(item)
    raw = mne.io.read_raw_edf(item)
    # Filter EEG channels ('P3' and 'C3')
    eeg_channels = [ch for ch in raw.info['ch_names'] if ch in channels]
    if len(eeg_channels) != len(channels):
        print(f"Error: channels not found in {item}. Skipping...")
        continue

    # Create a new Raw object with only EEG channels
    raw = raw.copy().pick_channels(eeg_channels)


    # Down Sampling
    raw = raw.copy().resample(sfreq=128)
    # Apply bandpass filter (example: 0.1 Hz - 40 Hz)
    raw.load_data()  # Load the data into memory
    raw.filter(l_freq=0.1, h_freq=40)
    #raw= raw.to_data_frame().to_numpy().astype(np.float32)

   # print("mean of data is  {:.6f}".format(np.mean(np.mean(raw))))
    # Segment data into epochs (e.g., 1-second epochs)
    events = mne.make_fixed_length_events(raw, duration=1.0)
    epochs = mne.Epochs(raw, events, tmin=0, tmax=1.0, baseline=None)

    X = epochs.get_data()
    # Standardize features (Z-score normalization)
    n_samples, n_channels, n_time_points = X.shape

    # Reshape to 2D (n_samples x (n_channels * n_time_points))
    #X_reshaped = X.reshape(n_samples, -1)
    X_reshaped=X
    # Apply StandardScaler

    # Compute mean and standard deviation along the time axis (axis 2)
    mean = np.mean(X_reshaped, axis=(0, 1, 2))
    std = np.std(X_reshaped, axis=(0, 1, 2))
    print("mean of data is  {:.6f}".format(np.mean(mean)))
     # Perform scaling
    eeg_data_scaled = (X_reshaped - mean) / std
    X_scaled = eeg_data_scaled

    all_eeg.append(X_scaled)


  #all_eeg=np.array(all_eeg)

  lb=LabelBinarizer()
  all_labels = pd.read_excel("/content/dataset/states.xlsx",usecols=["status"])
  all_labels=lb.fit_transform(all_labels)


  return all_eeg, all_labels, n_samples, n_channels ,n_time_points
  #return  X_train, X_test, y_train, y_test, n_samples, n_channels ,n_time_points

In [4]:
def create_vit_model(input_shape, num_classes):
    inputs = layers.Input(shape=input_shape, name="input_layer")

    # Patch creation
    patch_size = 4
    #patches = layers.Conv2D(filters=64, kernel_size=patch_size, strides=patch_size, activation='relu')(inputs)
    patches = layers.Conv2D(filters=16, kernel_size=patch_size, strides=patch_size, activation='relu')(inputs)

    # Flatten patches
    flattened_patches = layers.Flatten()(patches)

    # MLP head
    #mlp_output = layers.Dense(256, activation='gelu')(flattened_patches)
    mlp_output= layers.Dense(32, kernel_regularizer=regularizers.l2(0.01))(flattened_patches)
    mlp_output = BatchNormalization()(mlp_output)


    mlp_output = layers.Dropout(0.2)(mlp_output)

    # Classification head
    outputs = layers.Dense(num_classes, activation='softmax', name='output_layer')(mlp_output)

    model = models.Model(inputs=inputs, outputs=outputs)

    return model

In [5]:
class Patches(layers.Layer):
    def __init__(self, patch_size):
        super().__init__()
        self.patch_size = patch_size

    def call(self, images):
        input_shape = tf.shape(images)
        batch_size = input_shape[0]
        height = input_shape[1]
        width = input_shape[2]
        channels = input_shape[3]  # Assuming RGB images (3 channels)

        num_patches_h = height // self.patch_size
        num_patches_w = width // self.patch_size

        patches = tf.image.extract_patches(images, sizes=[1, self.patch_size, self.patch_size, 1], strides=[1, self.patch_size, self.patch_size, 1], rates=[1, 1, 1, 1], padding="VALID")
        patches = tf.reshape(patches, (batch_size, num_patches_h * num_patches_w, self.patch_size * self.patch_size * channels))

        return patches


In [6]:
def mlp(x, hidden_units, dropout_rate):
    for units in hidden_units:
        x = layers.Dense(units, activation="relu")(x)
        x = layers.Dropout(dropout_rate)(x)
    return x


In [7]:
def apply_augmentation(data, noise_level=0.2):
  np.random.seed(42)
  noise = np.random.normal(scale=noise_level, size=np.shape(data))
  augmented_data=data + noise

  return np.array(augmented_data)

In [8]:
def lr_scheduler(epoch, lr):
    if epoch < 5:
        return lr
    else:
        return lr * tf.math.exp(-0.1)


In [15]:
channels= ['T3', 'T5', 'T4', 'T6','Fp2','F3','Fz','Pz','C3','P3','O1','O2']
all_eeg, all_labels, n_samples, n_channels ,n_time_points = load_data()
#X_train, X_test, y_train, y_test, n_samples, n_channels ,n_time_points = load_data()


Folder dataset already exists
/content/dataset/22.edf
Extracting EDF parameters from /content/dataset/22.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 4225 samples (33.008 s)

Not setting metadata
1806 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 1806 events and 12

In [16]:
print( "n_samples= {} and  n_channels = {} and n_time_points= {}".format(n_samples, n_channels ,n_time_points))

n_samples= 1810 and  n_channels = 12 and n_time_points= 129


In [17]:
all_eeg=np.array(all_eeg)

<ipython-input-17-ae1a91f1323d>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  all_eeg=np.array(all_eeg)


In [18]:
for x in range(len(all_eeg)):
  all_eeg[x]=all_eeg[x][0:1802,0:12,0:129]
  #all_eeg[x]=all_eeg[x][0:1802,0:12,0:257]


In [19]:
all_eeg=np.stack( all_eeg, axis=0 )




In [20]:
k_folds = 5
kf = KFold(n_splits=k_folds, shuffle=True, random_state=42)
fold_accuracies = []

#X_train, X_test, y_train, y_test = train_test_split(all_eeg, all_labels, test_size=0.2, random_state=42)

In [ ]:
mlp_head_units =[16, 8] #=[32, 16]#[64, 32] [128, 64] #   [256, 128]
input_shape =np.shape(all_eeg[0])#  (1802, 12, 257)

In [ ]:
np.shape(all_eeg)

In [ ]:
lr_callback = LearningRateScheduler(lr_scheduler)


In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)


In [28]:
all_eeg_augmented = apply_augmentation(all_eeg)


In [22]:
import torch
import torch.nn as nn
from torch.nn import init
from torch import cosine_similarity
import torch.nn.functional as F


In [31]:


class Flatten(nn.Module):
    def forward(self, inputs):
        return inputs.view(inputs.size(0), -1)


class LayerNorm(nn.Module):
    r""" LayerNorm that supports two data formats: channels_last (default) or channels_first.
    The ordering of the dimensions in the inputs. channels_last corresponds to inputs with
    shape (batch_size, height, width, channels) while channels_first corresponds to inputs
    with shape (batch_size, channels, height, width).
    """

    def __init__(self, normalized_shape, eps=1e-6, data_format="channels_first"):
        super().__init__()
        self.weight = nn.Parameter(torch.ones(normalized_shape))
        self.bias = nn.Parameter(torch.zeros(normalized_shape))
        self.eps = eps
        self.data_format = data_format
        if self.data_format not in ["channels_last", "channels_first"]:
            raise NotImplementedError
        self.normalized_shape = (normalized_shape,)

    def forward(self, x):
        if self.data_format == "channels_last":
            return F.layer_norm(x, self.normalized_shape, self.weight, self.bias, self.eps)
        elif self.data_format == "channels_first":
            u = x.mean(1, keepdim=True)
            s = (x - u).pow(2).mean(1, keepdim=True)
            x = (x - u) / torch.sqrt(s + self.eps)
            x = self.weight[:, None, None] * x + self.bias[:, None, None]
            return x


class PatchEmbed(nn.Module):
    def __init__(self, args, in_c=1):
        super().__init__()

        self.proj1 = nn.Conv2d(in_c, 10, kernel_size=(17, 1), stride=(10, 1), padding=(8, 0))
        self.norm1 = LayerNorm(10, eps=1e-6, data_format="channels_first")
        self.relu = nn.ReLU(inplace=True)
        self.proj2 = nn.Conv2d(in_c, 6, kernel_size=(3, 1), stride=(1, 1), padding=(1, 0))
        self.pool = nn.MaxPool2d((10, 1))
        self.norm = LayerNorm(6, eps=1e-6, data_format="channels_first")    # for CHB-MIT

    def forward(self, x):   # (b, 1, t, n)
        x1 = self.relu(self.norm1(self.proj1(x)))
        x2 = self.pool(self.relu(self.proj2(x)))
        x2 = self.norm(x2)
        return torch.cat((x1, x2), dim=1)   # (b, 16, 128, c)


class Attention(nn.Module):
    def __init__(self, embed_dim, num_heads=1, bias=True):  # 128 2 12 0
        super(Attention, self).__init__()
        self.num_heads = num_heads
        self.head_dim = embed_dim // num_heads
        self.to_qkv = nn.Conv2d(embed_dim, 2 * embed_dim + num_heads, 1, 1, bias=bias)
        self.attend = nn.Softmax(dim=-1)

        self.to_out = nn.Sequential(
            nn.Conv2d(embed_dim, embed_dim, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),  #
            nn.BatchNorm2d(embed_dim),
            nn.ReLU(inplace=True),
        )

    def forward(self, x):
        b, c, t, n = x.shape  # (b,c,t,n)
        qkv = self.to_qkv(x).reshape(b, self.num_heads, -1, t, n)
        i, k, v = torch.split(
            qkv, split_size_or_sections=[1, self.head_dim, self.head_dim], dim=2
        )
        scores = self.attend(i)  # (b, h, 1, t, n)
        attn_vector = k * scores  # (b, h, c//h, t, n)
        attn_vector = torch.sum(attn_vector, dim=-1, keepdim=True)  # (b, h, c//h, t, 1)
        out = (F.relu(v) * attn_vector.expand_as(v)).reshape(b, -1, t, n)  # (b, c, t, n)
        out = self.to_out(out)
        return out


class PreNorm(nn.Module):
    def __init__(self, dim, fn):
        super(PreNorm, self).__init__()
        self.norm = nn.BatchNorm2d(dim)
        self.fn = fn

    def forward(self, x, **kwargs):
        return self.fn(self.norm(x), **kwargs)


class FeedForward(nn.Module):
    def __init__(self, dim, mlp_ratio):
        super(FeedForward, self).__init__()
        self.net = nn.Sequential(
            nn.Conv2d(dim, dim, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),  #
            nn.BatchNorm2d(dim),
            nn.ReLU(inplace=True),
        )

    def forward(self, x):   # (64,32,100,7)
        return self.net(x)


class Former(nn.Module):
    def __init__(self, dim, num_heads, mlp_ratio):
        super(Former, self).__init__()
        self.layers = nn.ModuleList([])
        self.layers.append(nn.ModuleList([
            PreNorm(dim, Attention(dim, num_heads)),
            PreNorm(dim, FeedForward(dim, mlp_ratio))
        ]))

    def forward(self, x):  # z=(64,h,d)
        for attn, ff in self.layers:
            x = attn(x) + x
            x = ff(x) + x
        return x


class BaseBlock(nn.Module):
    def __init__(self, inp=16, out=32, ks=3, stride=2, num_heads=1, mlp_ratio=2):
        super(BaseBlock, self).__init__()
        self.mlp_ratio = mlp_ratio
        self.former = Former(dim=out, num_heads=num_heads, mlp_ratio=mlp_ratio)

        self.downsample_layer = nn.Sequential(
            nn.Conv2d(inp, inp*4, ks, stride, padding=ks//2, groups=inp),
            nn.ReLU(inplace=True),
            nn.Conv2d(inp*4, out, 1, 1, 0, bias=False),  # don't need bias
            nn.BatchNorm2d(out),
        )
        self.downsample = nn.Sequential(
            nn.Conv2d(out, out * 2, 3, 1, padding=1, groups=out),
            nn.ReLU(inplace=True),
            nn.Conv2d(out * 2, out, 1, 1, 0, bias=False),  # don't need bias
            nn.BatchNorm2d(out),
        )

    def forward(self, x):
        x = self.downsample_layer(x)
        # x = self.downsample(x)
        x = self.former(x)
        return x


class HViT(nn.Module):
    def __init__(self, args, cfg, inputs_shape):
        super().__init__()
        self.w1 = torch.nn.Parameter(torch.FloatTensor(1), requires_grad=True)
        # initialization
        self.w1.data.fill_(8)

        self.args = args
        self.patchembed = PatchEmbed(args, in_c=1)
        for arg in cfg['C2T1']:
            self.C2Tblock1 = BaseBlock(**arg)
        for arg in cfg['C2T2']:
            self.C2Tblock2 = BaseBlock(**arg)

        self.mu_proj = nn.Sequential(
            nn.Conv2d(32, 32 * 4, 3, 2, padding=1, groups=32),
            nn.ReLU(inplace=True),
            nn.Conv2d(32 * 4, 64, 1, 1, 0, bias=False),  # don't need bias
            nn.BatchNorm2d(64),
            nn.Conv2d(64, 64, kernel_size=(5, 3), stride=(1, 1), padding=(2, 0)),
            nn.BatchNorm2d(64),
            nn.GELU(),
            nn.AdaptiveAvgPool2d((1, 1)),
            Flatten()
        )

        self.sigma_proj = nn.Sequential(
            nn.Conv2d(32, 32 * 4, 3, 2, padding=2, groups=32),
            nn.ReLU(inplace=True),
            nn.Conv2d(32 * 4, 64, 1, 1, 0, bias=False),
            nn.BatchNorm2d(64),
            nn.AdaptiveAvgPool2d((1, 1)),
            Flatten()
        )

        self.init_params()

    def Gauss_reparameterize(self, mu, logvar):  # Gaussian
        std = torch.exp(logvar).sqrt()
        epsilon = torch.randn_like(std)
        return mu + epsilon * std

    def Laplace_reparameterize(self, mu, logb):
        b = torch.exp(logb).clamp(min=1e-8)
        epsilon = torch.randn_like(mu)
        z = torch.rand_like(mu)
        sign = torch.sign(z - 0.5)
        epsilon = sign * epsilon
        epsilon = torch.where(torch.abs(z - 0.5) > 1e-8, epsilon, torch.zeros_like(mu))
        mask = (1 - torch.abs(epsilon)) > 1e-8
        epsilon = torch.where(mask, epsilon, torch.zeros_like(mu))
        z = torch.where(mask, z, torch.zeros_like(mu))
        t = 2.0 * torch.abs(z - 0.5) + 1e-8
        t = torch.where(t < 1, t, torch.full_like(t, 1))
        return mu - b * sign * torch.log1p(-t * epsilon)

    def Gauss_kL_loss(self, mu, logvar):
        kl_loss = -0.5 * (1 + logvar - mu.pow(2) - logvar.exp()).sum(dim=1).mean()
        return kl_loss

    def Laplace_kl_loss(self, mu, logb):
        kl_loss = - (1 + logb - mu.abs() - torch.exp(logb)).sum(dim=1).mean()
        return kl_loss

    def sigma_loss(self, log_sigma):
        sigma = torch.exp(log_sigma)
        avg_sigma = sigma.mean(dim=1, keepdim=True).repeat(1, log_sigma.shape[1])
        sigma_loss = torch.abs(sigma / avg_sigma - 1.0).mean()
        return sigma_loss

    def forward(self, x, target_y, classifier, phase='train'):
        x = self.patchembed(x)  # (128, 16, 128, 22)
        x = self.C2Tblock1(x)
        x = self.C2Tblock2(x)

        if self.args.pred_mode == 'Base':
            features = self.mu_proj(x)  # (128,64)
            logits = classifier(features)   # (128, 2)
            loss = F.cross_entropy(logits, target_y).cuda()
            return logits, loss
        else:
            mu = self.mu_proj(x)  # (128,64,1,1)
            log_sigma = self.sigma_proj(x)

            if phase == 'train':
                rep_features = self.Gauss_reparameterize(mu, log_sigma) if self.args.pred_mode == 'DUL-G' \
                    else self.Laplace_reparameterize(mu, log_sigma)
                rep_logits = classifier(rep_features)
                ce_loss = F.cross_entropy(rep_logits, target_y)
                kl_loss = self.Gauss_kL_loss(mu, log_sigma)if self.args.pred_mode == 'DUL-G' \
                    else self.Laplace_kl_loss(mu, log_sigma)

                loss = (ce_loss * torch.sigmoid(self.w1) + (1 - torch.sigmoid(self.w1)) * kl_loss).cuda()
                # print(f'{torch.sigmoid(self.w1).item()}')
                return rep_logits, loss
            else:
                logits = classifier(mu)
                probs = F.softmax(logits, dim=-1)
                uncertainty = -(probs * torch.log(probs + 1e-16)).sum(-1)
                ce_loss = F.cross_entropy(logits, target_y)

                return logits, ce_loss, uncertainty

    def init_params(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                init.constant_(m.weight, 1)
                init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                init.normal_(m.weight, std=0.001)  # 0.001
                if m.bias is not None:
                    init.constant_(m.bias, 0)


class Classifier(nn.Module):
    def __init__(self, feature_size=64):
        super(Classifier, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(feature_size, 2)
        )

    def forward(self, x):
        return self.fc(x)

In [32]:
for fold, (train_index, test_index) in enumerate(kf.split(all_eeg_augmented)):
    print(f"Fold {fold+1}/{k_folds}")

    # Split data into training and testing sets
    X_train, X_test = all_eeg_augmented[train_index], all_eeg_augmented[test_index]
    y_train, y_test = all_labels[train_index], all_labels[test_index]

    # Apply data preprocessing if needed


    args=""
    config_my=[]

    model = HViT(args, config_my, X_train.shape).cuda()


    # Define and compile your model
    vit_model = create_vit_model(input_shape, 2)
    optimizer = keras.optimizers.Adam(learning_rate=0.001)
    vit_model.compile(optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=["accuracy"])

    #print shape of data
    print("Shapes of training data and labels:")
    print("X_train shape:", X_train.shape)
    print("y_train shape:", y_train.shape)

    print("Shapes of testing data and labels:")
    print("X_test shape:", X_test.shape)
    print("y_test shape:", y_test.shape)
    # Train the model
    history = vit_model.fit(X_train, y_train, batch_size=16, epochs=20, validation_data=(X_test, y_test), verbose=0)

    # Evaluate the model on the test set
    test_loss, test_accuracy = vit_model.evaluate(X_test, y_test)
    fold_accuracies.append(test_accuracy)
    print(f"Test Accuracy for Fold {fold+1}: {test_accuracy:.4f}")

# Calculate and print the average accuracy across all folds
avg_accuracy = sum(fold_accuracies) / len(fold_accuracies)
print(f"Average Test Accuracy across {k_folds} folds: {avg_accuracy:.4f}")

Fold 1/5


TypeError: list indices must be integers or slices, not str

In [ ]:

loo = LeaveOneOut()

# Initialize lists to store evaluation metrics for each fold
fold_accuracies = []

# Iterate over each fold
for fold, (train_index, test_index) in enumerate(loo.split(all_eeg_augmented)):
    print(f"Fold {fold+1}/{len(all_eeg_augmented)}")

    # Split data into training and testing sets
    X_train, X_test = all_eeg_augmented[train_index], all_eeg_augmented[test_index]
    y_train, y_test = all_labels[train_index], all_labels[test_index]

    # Apply data preprocessing if needed

    # Define and compile your model
    vit_model = create_vit_model(input_shape, 2)
    optimizer = keras.optimizers.Adam(learning_rate=0.001)
    vit_model.compile(optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=["accuracy"])

    # Train the model
    history = vit_model.fit(X_train, y_train, batch_size=16, epochs=20, validation_data=(X_test, y_test), verbose=0)

    # Evaluate the model on the test set
    test_loss, test_accuracy = vit_model.evaluate(X_test, y_test)
    fold_accuracies.append(test_accuracy)
    print(f"Test Accuracy for Fold {fold+1}: {test_accuracy:.4f}")

# Calculate and print the average accuracy across all folds
avg_accuracy = sum(fold_accuracies) / len(fold_accuracies)
print(f"Average Test Accuracy across {len(all_eeg)} folds: {avg_accuracy:.4f}")
